# Processing the Data

The goal is to treat both datasets for missing values with the end goal being to merge both datasets to 1 processed CSV file

In [1]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import requests
import os
from datetime import datetime
from dotenv import load_dotenv

In [2]:
aqi_data = pd.read_csv("../data/raw/AQI-Data.csv", parse_dates=["Date"])
weather_data = pd.read_csv("../data/raw/weather-data.csv", parse_dates=["date"])

## Reading and parsing AQI data

In [3]:
aqi_data[:10]

,Unnamed: 0,Date,AQI
0,0,2018-01-01,75.0
1,1,2018-01-02,76.0
2,2,2018-01-03,79.0
3,3,2018-01-04,84.0
4,4,2018-01-05,97.0
5,5,2018-01-06,84.0
6,6,2018-01-07,63.0
7,7,2018-01-08,65.0
8,8,2018-01-09,78.0
9,9,2018-01-10,104.0


In [4]:
len(aqi_data)

2557

In [5]:
aqi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  2557 non-null   int64         
 1   Date        2557 non-null   datetime64[ns]
 2   AQI         2303 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 60.1 KB


In [6]:
aqi_data.describe()

,Unnamed: 0,Date,AQI
count,2557.000000,2557,2303.000000
mean,1278.000000,2021-07-02 00:00:00,102.771168
min,0.000000,2018-01-01 00:00:00,19.000000
25%,639.000000,2019-10-02 00:00:00,68.000000
50%,1278.000000,2021-07-02 00:00:00,94.000000
75%,1917.000000,2023-04-02 00:00:00,128.000000
max,2556.000000,2024-12-31 00:00:00,435.000000
std,738.286643,NaN,44.665376


In [7]:
aqi_data.isna().sum()

Unnamed: 0      0
Date            0
AQI           254
dtype: int64

In [8]:
aqi_data.dtypes

Unnamed: 0             int64
Date          datetime64[ns]
AQI                  float64
dtype: object

## Treating missing values

To treat missing values we will do the following steps
* 1. Create columns for month and year
  2. Fill missing AQI values using the median value of the month of the given year

In [9]:
# Add month and year to each value in the dataframe

aqi_data["Year"] = aqi_data["Date"].dt.year
aqi_data["Month"] = aqi_data["Date"].dt.month

In [10]:
aqi_data

,Unnamed: 0,Date,AQI,Year,Month
0,0,2018-01-01,75.0,2018,1
1,1,2018-01-02,76.0,2018,1
2,2,2018-01-03,79.0,2018,1
3,3,2018-01-04,84.0,2018,1
4,4,2018-01-05,97.0,2018,1
...,...,...,...,...,...
2552,2552,2024-12-27,109.0,2024,12
2553,2553,2024-12-28,95.0,2024,12
2554,2554,2024-12-29,110.0,2024,12
2555,2555,2024-12-30,87.0,2024,12


In [11]:
#Using grouping to fill missing values

groups = aqi_data.groupby(["Year", "Month"])
medians = groups["AQI"].transform("median")
aqi_data["AQI"] = aqi_data["AQI"].fillna(medians)

In [12]:
aqi_data

,Unnamed: 0,Date,AQI,Year,Month
0,0,2018-01-01,75.0,2018,1
1,1,2018-01-02,76.0,2018,1
2,2,2018-01-03,79.0,2018,1
3,3,2018-01-04,84.0,2018,1
4,4,2018-01-05,97.0,2018,1
...,...,...,...,...,...
2552,2552,2024-12-27,109.0,2024,12
2553,2553,2024-12-28,95.0,2024,12
2554,2554,2024-12-29,110.0,2024,12
2555,2555,2024-12-30,87.0,2024,12


In [13]:
aqi_data.isna().sum()

Unnamed: 0     0
Date           0
AQI           31
Year           0
Month          0
dtype: int64

In [14]:
# Check for data values that are still missing

mask = aqi_data["AQI"].isna()
aqi_data[mask]

,Unnamed: 0,Date,AQI,Year,Month
1673,1673,2022-08-01,NaN,2022,8
1674,1674,2022-08-02,NaN,2022,8
1675,1675,2022-08-03,NaN,2022,8
1676,1676,2022-08-04,NaN,2022,8
1677,1677,2022-08-05,NaN,2022,8
1678,1678,2022-08-06,NaN,2022,8
1679,1679,2022-08-07,NaN,2022,8
1680,1680,2022-08-08,NaN,2022,8
1681,1681,2022-08-09,NaN,2022,8
1682,1682,2022-08-10,NaN,2022,8


In [15]:
# Filling in missing values in august by taking median of values from the respective day in August or other years in the dataset

# Start by adding a day column
aqi_data["Day"] = aqi_data["Date"].dt.day

In [16]:
aqi_data

,Unnamed: 0,Date,AQI,Year,Month,Day
0,0,2018-01-01,75.0,2018,1,1
1,1,2018-01-02,76.0,2018,1,2
2,2,2018-01-03,79.0,2018,1,3
3,3,2018-01-04,84.0,2018,1,4
4,4,2018-01-05,97.0,2018,1,5
...,...,...,...,...,...,...
2552,2552,2024-12-27,109.0,2024,12,27
2553,2553,2024-12-28,95.0,2024,12,28
2554,2554,2024-12-29,110.0,2024,12,29
2555,2555,2024-12-30,87.0,2024,12,30


In [23]:
# Computing medians for every day in August
august_medians = (aqi_data[(aqi_data["Month"]==8) & (aqi_data["Year"] != 2022)]
    .groupby("Day")["AQI"]
    .median()
                 )

In [24]:
august_medians

Day
1     72.0
2     72.0
3     51.5
4     54.5
5     54.0
6     63.0
7     59.5
8     57.5
9     54.5
10    58.5
11    59.5
12    57.0
13    59.5
14    61.5
15    59.0
16    65.5
17    63.0
18    67.0
19    66.5
20    63.5
21    68.0
22    62.5
23    58.5
24    63.5
25    61.0
26    63.5
27    65.0
28    61.5
29    65.0
30    71.0
31    62.5
Name: AQI, dtype: float64

In [25]:
mask = (aqi_data["Year"] == 2022) & (aqi_data["Month"] == 8) & (aqi_data["AQI"].isna())

In [26]:
mask

0       False
1       False
2       False
3       False
4       False
        ...  
2552    False
2553    False
2554    False
2555    False
2556    False
Length: 2557, dtype: bool

In [27]:
aqi_data.loc[mask, "AQI"] = aqi_data.loc[mask, "Day"].map(august_medians)

In [28]:
aqi_data.isna().sum()

Unnamed: 0    0
Date          0
AQI           0
Year          0
Month         0
Day           0
dtype: int64

## Reading and Parsing Weather Data

In [37]:
weather_data

,Unnamed: 0,date,temperature_2m_max,temperature_2m_min,precipitation_sum,weather_code,sunshine_duration,cloud_cover_mean,wind_speed_10m_mean,winddirection_10m_dominant
0,0,2015-01-11 00:00:00+00:00,28.209,13.259,0.0,0.0,37052.594,0.000000,8.084107,87.986560
1,1,2015-01-12 00:00:00+00:00,27.709,12.509,0.0,3.0,37158.527,21.750000,7.937115,89.543495
2,2,2015-01-13 00:00:00+00:00,28.159,14.859,0.0,3.0,37167.082,48.250000,6.330598,58.540066
3,3,2015-01-14 00:00:00+00:00,28.959,15.459,0.0,3.0,37175.984,43.041668,5.552982,51.170260
4,4,2015-01-15 00:00:00+00:00,28.209,14.009,0.0,3.0,37290.023,44.416668,5.279206,94.532120
...,...,...,...,...,...,...,...,...,...,...
3638,3638,2024-12-27 00:00:00+00:00,30.253,20.053,2.3,53.0,31781.553,59.875000,6.546581,169.180630
3639,3639,2024-12-28 00:00:00+00:00,30.103,19.503,0.2,51.0,36000.000,21.458334,5.255987,268.085020
3640,3640,2024-12-29 00:00:00+00:00,31.003,19.603,0.2,51.0,34506.890,35.708332,4.277875,57.926180
3641,3641,2024-12-30 00:00:00+00:00,31.053,19.803,0.0,0.0,36073.367,1.791667,3.847927,62.987446


In [38]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3643 entries, 0 to 3642
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   Unnamed: 0                  3643 non-null   int64              
 1   date                        3643 non-null   datetime64[ns, UTC]
 2   temperature_2m_max          3643 non-null   float64            
 3   temperature_2m_min          3643 non-null   float64            
 4   precipitation_sum           3643 non-null   float64            
 5   weather_code                3643 non-null   float64            
 6   sunshine_duration           3643 non-null   float64            
 7   cloud_cover_mean            3643 non-null   float64            
 8   wind_speed_10m_mean         3643 non-null   float64            
 9   winddirection_10m_dominant  3643 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(8), int64(1)
memory usage

In [40]:
weather_data.describe()

,Unnamed: 0,temperature_2m_max,temperature_2m_min,precipitation_sum,weather_code,sunshine_duration,cloud_cover_mean,wind_speed_10m_mean,winddirection_10m_dominant
count,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000
mean,1821.000000,30.922648,20.245362,3.352265,26.948394,34049.086598,48.265166,10.018777,215.794624
std,1051.787843,4.040142,3.107583,8.984942,27.248104,10452.299743,36.094136,4.853772,87.731761
min,0.000000,20.203000,9.403000,0.000000,0.000000,0.000000,0.000000,2.647810,0.260454
25%,910.500000,28.003000,18.053000,0.000000,2.000000,33882.047000,13.854166,6.257000,107.119605
50%,1821.000000,29.903000,21.353000,0.000000,3.000000,37325.277000,43.541668,8.452968,256.334200
75%,2731.500000,33.806000,22.603000,2.500000,53.000000,39809.224500,84.791668,12.908714,276.041100
max,3642.000000,42.953000,27.653000,174.299990,65.000000,43200.000000,100.000000,34.238230,359.999970


In [33]:
weather_data.isna().sum()

Unnamed: 0                    0
date                          0
temperature_2m_max            0
temperature_2m_min            0
precipitation_sum             0
weather_code                  0
sunshine_duration             0
cloud_cover_mean              0
wind_speed_10m_mean           0
winddirection_10m_dominant    0
dtype: int64

### Weather data has no missing values

## Merging both datasets 

In [41]:
# Normalising date timeones (weather is in UTC)

weather_data["date"] = weather_data["date"].dt.tz_localize(None)
aqi_data["Date"] = aqi_data["Date"].dt.tz_localize(None)

In [43]:
# Merge both datasets with an inner join

merged = aqi_data.merge(weather_data,
                        left_on = "Date",
                        right_on= "date",
                        how= "inner")

In [44]:
merged

,Unnamed: 0_x,Date,AQI,Year,Month,Day,Unnamed: 0_y,date,temperature_2m_max,temperature_2m_min,precipitation_sum,weather_code,sunshine_duration,cloud_cover_mean,wind_speed_10m_mean,winddirection_10m_dominant
0,0,2018-01-01,75.0,2018,1,1,1086,2018-01-01,28.253,14.503,0.0,0.0,36062.707,0.000000,5.679092,270.613800
1,1,2018-01-02,76.0,2018,1,2,1087,2018-01-02,28.453,12.903,0.0,1.0,36040.188,7.166666,4.744073,278.325560
2,2,2018-01-03,79.0,2018,1,3,1088,2018-01-03,28.903,15.303,0.0,0.0,36101.605,0.083333,5.611268,285.883900
3,3,2018-01-04,84.0,2018,1,4,1089,2018-01-04,28.353,15.303,0.0,0.0,36083.652,1.541667,5.784296,326.564940
4,4,2018-01-05,97.0,2018,1,5,1090,2018-01-05,28.453,14.003,0.0,3.0,36066.500,46.125000,5.506170,287.776520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2552,2024-12-27,109.0,2024,12,27,3638,2024-12-27,30.253,20.053,2.3,53.0,31781.553,59.875000,6.546581,169.180630
2553,2553,2024-12-28,95.0,2024,12,28,3639,2024-12-28,30.103,19.503,0.2,51.0,36000.000,21.458334,5.255987,268.085020
2554,2554,2024-12-29,110.0,2024,12,29,3640,2024-12-29,31.003,19.603,0.2,51.0,34506.890,35.708332,4.277875,57.926180
2555,2555,2024-12-30,87.0,2024,12,30,3641,2024-12-30,31.053,19.803,0.0,0.0,36073.367,1.791667,3.847927,62.987446


In [46]:
# Dropping unnecessary columns

merged = merged.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y", "date"])

In [47]:
# Set Date as Index
merged.set_index("Date", inplace=True)

In [48]:
merged

,AQI,Year,Month,Day,temperature_2m_max,temperature_2m_min,precipitation_sum,weather_code,sunshine_duration,cloud_cover_mean,wind_speed_10m_mean,winddirection_10m_dominant
Date,,,,,,,,,,,,
2018-01-01,75.0,2018,1,1,28.253,14.503,0.0,0.0,36062.707,0.000000,5.679092,270.613800
2018-01-02,76.0,2018,1,2,28.453,12.903,0.0,1.0,36040.188,7.166666,4.744073,278.325560
2018-01-03,79.0,2018,1,3,28.903,15.303,0.0,0.0,36101.605,0.083333,5.611268,285.883900
2018-01-04,84.0,2018,1,4,28.353,15.303,0.0,0.0,36083.652,1.541667,5.784296,326.564940
2018-01-05,97.0,2018,1,5,28.453,14.003,0.0,3.0,36066.500,46.125000,5.506170,287.776520
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,109.0,2024,12,27,30.253,20.053,2.3,53.0,31781.553,59.875000,6.546581,169.180630
2024-12-28,95.0,2024,12,28,30.103,19.503,0.2,51.0,36000.000,21.458334,5.255987,268.085020
2024-12-29,110.0,2024,12,29,31.003,19.603,0.2,51.0,34506.890,35.708332,4.277875,57.926180


In [49]:
# Export Merged Dataset
merged.to_csv("../data/processed/processed.csv")